In [1]:
# Let's build a Gaussian Mixture Variational Autoencoder (GMVAE) in PyTorch
# We'll use the MNIST dataset
# We'll use a Mixture of Gaussians as prior and Gaussian posterior

# Import libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
from torch.distributions import Normal, Multinomial, kl_divergence


The generative model is $p_{\beta, \theta} = p(w)p(z)p_{\beta}(x|w,z)p_{\theta}(y|x)$

where: \
$w \sim N(0, Id)$ \
$z \sim Mult(\pi)$ \
$x|z,x \sim \prod_{k=1}^{K} N(\mu_{z_k})(w;beta), diag(\sigma^2_{z_k})(w;\beta)))^{z_k}$ \
$y|x \sim N(\mu(x;\theta), diag(\sigma^2(x; \theta)))$

where $\mu_{z_k}(,\beta), \ \sigma^2_{z_k}(,\beta), \ \mu(,\theta), \ \sigma^2(,\theta)$ are given by neural networks with parameters beta and theta respectively.
That is, the observed
sample y is generated from a neural network observation model parametrised by theta and the contin- uous latent variable x. Furthermore, the distribution of x|w is a Gaussian mixture with means and variances specified by another neural network model parametrised by beta and with input w.

In [ ]:
# define the model

class GMVAE(nn.Module):
    def __init__(self, INPUT_DIM, H_DIM, Z_DIM, num_clusters):
        super(GMVAE, self).__init__()
        self.INPUT_DIM = INPUT_DIM
        self.H_DIM = H_DIM
        self.Z_DIM = Z_DIM
        self.num_clusters = num_clusters

        # encoder
        pass

        # decoder
        pass

        # prior
        pass

    def encode(self, x):
        # encoder
        pass

        # posterior
        pass

        return mu, sigma, pi

    def decode(self, z):
        # decoder
        pass

        # likelihood
        pass    

    def forward(self, x):
        # encode
        w = Normal(torch.zeros(self.num_clusters, self.Z_DIM), torch.ones(self.num_clusters, self.Z_DIM)).sample()
        mu, sigma, pi = self.encode(x)
        y = Multinomial(1, pi).sample()


        # decode
        pass

        # prior
        pass

        # posterior
        pass

        # likelihood
        pass

        # KL divergence
        pass

        # ELBO
        pass

        # loss
        pass

        return 
   
    def reparameterize(self, mu, sigma, w):
        eps_z = torch.randn_like(sigma)
        eps_w = torch.randn_like(w)
        z = mu + eps_z * torch.sqrt(sigma)
        w = w + eps_w * torch.sqrt(torch.ones_like(w))
        return z, w    






In [10]:
Multinomial(1, torch.tensor([0.2,0.8])).sample()

tensor([0., 1.])